In [ ]:
import os
os.chdir("../")
os.getcwd()

In [ ]:
from attacker.query import *
from attacker.utils import *

set_seed(42)

# REF: https://towardsdev.com/implement-resnet-with-pytorch-a9fb40a77448

# Query victim model & Train attacker model

Uses default configuration info

In [ ]:
# get dataset in dataloader
trainloader, testloader, outputs = getDataset(config["victim"]["data"])

# query & save test data
querytestloader = QueryVictim(config["victim"], outputs, testloader, len(testloader.dataset))

# query train data
querytrainloader = QueryVictim(config["victim"], outputs, trainloader, config["query_size"], query_type=config["query_type"])

# initialize attacker model
attacker = getModel(config["attacker"], outputs)

# train attacker model
attacker_result = Training(attacker, querytrainloader, querytestloader, (-1, 3, 32, 32), config["epochs"], 
                           optimizer=torch.optim.Adam(attacker.parameters(), lr=config["learning_rate"]),
                           loss=torch.nn.CrossEntropyLoss())

# save & visualize model inference
title = f'Trial-A_{config["attacker"]}_{config["victim"]["model_name"]}_{config["victim"]["data"]}{config["query_type"]}_{config["query_size"]}'
percent = SaveVisualize(attacker, attacker_result,title )
results_dict={"Victim":config["victim"]["model_name"],
                "Dataset":config["victim"]["data"],
                "QueryType": config["query_type"],
                "QuerySize": config["query_size"],
                "Attacker": 'resnet34',
                "Train Loss": percent[0],
                "Train Accuracy": percent[1],
                "Test Loss": percent[2],
                "Test Accuracy": percent[3]}
print(results_dict)

# Investigations

In [ ]:
Results = []
# Iterate through Victim Model & Dataset
for victim in parameters["victim"]:
    print('---------------------------------------------------------------------------')
    # get dataset in dataloader
    trainloader, testloader, outputs = getDataset(victim["data"])

    # query test data
    querytestloader = QueryVictim(victim, outputs, testloader, len(testloader.dataset))
    
    # Iterate through Attacker Model
    for attacker_type in parameters["attacker"]:
        # Iterate Through Query Type
        for querytype in parameters["query_type"]:
            # Iterate Through Query Size
            for size in parameters["query_size"]:
                print('-----------------------------------------------------------------------------')
                print(f'-----------------------Dataset: {victim["data"]}----------------------------')
                print(f'--------Victim: {victim["model_name"]} Attacker: {attacker_type}------------')
                print(f'---------------Query Type: {querytype} Query Size: {size}-------------------')

                # query train data
                querytrainloader = QueryVictim(victim, outputs, trainloader, size, query_type=querytype)

                # initialize attacker model
                attacker = getModel(attacker_type, outputs)

                # train attacker model
                attacker_result = Training(attacker, querytrainloader, querytestloader, (-1, 3, 32, 32), config["epochs"], 
                                           optimizer=torch.optim.Adam(attacker.parameters(), lr=config["learning_rate"]),
                                           loss=torch.nn.CrossEntropyLoss())

                # save & visualize model inference
                title = f'A_{attacker_type}_{victim["model_name"]}_{victim["data"]}_{querytype}_{size}'
                percent = SaveVisualize(attacker, attacker_result, title)
                results_dict={"Victim":victim["model_name"],
                                "Dataset":victim["data"],
                                "QueryType": querytype,
                                "QuerySize": size,
                                "Attacker": attacker_type,
                                "Train Loss": percent[0],
                                "Train Accuracy": percent[1],
                                "Test Loss": percent[2],
                                "Test Accuracy": percent[3]}

                print(results_dict)
                Results.append(results_dict)
                print('-----------------------------------------------------------------------------')